## Imports

In [46]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights

## Data preparation

In [73]:
from download_rps import download_rps

data_path = "../data/"
train_data_path = data_path + "rps"
val_data_path = data_path + "rps-test-set"
download_rps(data_path)
class_names = ["paper", "rock", "scissors"]

rps folder already exists!
rps-test-set folder already exists!


In [48]:
# ImageNet statistics
normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transformer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])

# Builds a dataset of each set
train_data = ImageFolder(root=train_data_path, transform=transformer)
val_data = ImageFolder(root=val_data_path, transform=transformer)

# Builds a loader of each set
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

## Feature extraction

### Model configuration

In [49]:
def freeze_model(model):
    for parameter in model.parameters():
        parameter.requires_grad = False

# Set the seed
torch.manual_seed(42)

# Load the model
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)

# Change the top layer to Identity
resnet.fc = nn.Identity()
# Freeze the model
freeze_model(resnet)

### Preprocess data

In [50]:
def preprocess_dataset(model, dataset):
    features = torch.Tensor()
    labels = torch.Tensor()
    for x, y in dataset:
        model.eval()
        features = torch.cat([features, model(x)])
        labels = torch.cat([labels, y])

    dataset = TensorDataset(features, labels)
    return dataset

# Preprocess the data
train_preproc = preprocess_dataset(resnet, train_loader)
val_preproc = preprocess_dataset(resnet, val_loader)

### Save features

In [51]:
train_preproc_path = data_path + "train_preproc.pth"
val_preproc_path = data_path + "val_preproc.pth"
torch.save(train_preproc.tensors, train_preproc_path)
torch.save(val_preproc.tensors, val_preproc_path)

### Load features

In [52]:
train_preproc_path = data_path + "train_preproc.pth"
val_preproc_path = data_path + "val_preproc.pth"
train_preproc_data = TensorDataset(*torch.load(train_preproc_path))
val_preproc_data = TensorDataset(*torch.load(val_preproc_path))
train_preproc_loader = DataLoader(train_preproc_data, batch_size=16, shuffle=True)
val_preproc_loader = DataLoader(val_preproc_data, batch_size=16)

## Top model

### Model configuration

In [53]:
torch.manual_seed(42)
top_model = nn.Sequential(nn.Linear(512, 3))
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_top = optim.Adam(top_model.parameters(), lr=3e-4)

### Model training and evaluation

In [81]:
import numpy as np

def evaluate(model, data_loader):
    model.eval()
    n_dims = 0
    with torch.no_grad():
        for x, y in data_loader:
            y_pred = model(x)
            _, n_dims = y_pred.shape
            break
    true_positives = np.zeros(n_dims)
    false_positives = np.zeros(n_dims)
    true_negatives = np.zeros(n_dims)
    false_negatives = np.zeros(n_dims)
    with torch.no_grad():
        for x, y in data_loader:
            y_pred = model(x)
            _, predicted = torch.max(y_pred, 1)

            for c in range(n_dims):
                true_positives[c] += (predicted[y == c] == c).sum().item()
                false_positives[c] += (predicted[y != c] == c).sum().item()
                false_negatives[c] += (predicted[y == c] != c).sum().item()
                true_negatives[c] += (predicted[y != c] != c).sum().item()

    for i in range(n_dims):
        precision = true_positives[i] / (true_positives[i] + false_positives[i])
        recall = true_positives[i] / (true_positives[i] + false_negatives[i])
        f1 = 2 * (precision * recall) / (precision + recall)
        print(f"\n{class_names[i].capitalize()}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1: {f1:.4f}")

def train(model, train_loader, val_loader, loss_fn, optimizer, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            y_pred = model(x)
            loss = loss_fn(y_pred, y.long())
            loss.backward()
            optimizer.step()

        print(f"\nEPOCH {epoch + 1}")
        evaluate(model, val_loader)


EPOCH 1

Paper
Precision: 0.9589
Recall: 0.5645
F1: 0.7107

Rock
Precision: 0.7126
Recall: 1.0000
F1: 0.8322

Scissors
Precision: 0.8400
Recall: 0.8468
F1: 0.8434

EPOCH 2

Paper
Precision: 0.9730
Recall: 0.5806
F1: 0.7273

Rock
Precision: 0.7045
Recall: 1.0000
F1: 0.8267

Scissors
Precision: 0.8607
Recall: 0.8468
F1: 0.8537

EPOCH 3

Paper
Precision: 0.9620
Recall: 0.6129
F1: 0.7488

Rock
Precision: 0.7251
Recall: 1.0000
F1: 0.8407

Scissors
Precision: 0.8607
Recall: 0.8468
F1: 0.8537

EPOCH 4

Paper
Precision: 0.9855
Recall: 0.5484
F1: 0.7047

Rock
Precision: 0.7006
Recall: 1.0000
F1: 0.8239

Scissors
Precision: 0.8333
Recall: 0.8468
F1: 0.8400

EPOCH 5

Paper
Precision: 0.9589
Recall: 0.5645
F1: 0.7107

Rock
Precision: 0.7126
Recall: 1.0000
F1: 0.8322

Scissors
Precision: 0.8400
Recall: 0.8468
F1: 0.8434

EPOCH 6

Paper
Precision: 0.9714
Recall: 0.5484
F1: 0.7010

Rock
Precision: 0.7045
Recall: 1.0000
F1: 0.8267

Scissors
Precision: 0.8333
Recall: 0.8468
F1: 0.8400

EPOCH 7

Paper


## Using the original dataset

### Reattach the top model

### Training

In [ ]:
train(top_model, train_preproc_loader, val_preproc_loader, multi_loss_fn, optimizer_top, 11)

### Evaluation

In [82]:
resnet.fc = top_model
evaluate(resnet, val_loader)


Paper
Precision: 0.9605
Recall: 0.5887
F1: 0.7300

Rock
Precision: 0.7126
Recall: 1.0000
F1: 0.8322

Scissors
Precision: 0.8607
Recall: 0.8468
F1: 0.8537
